In [1]:
ljy_fp = r"C:\Users\13377\Desktop\hw-memory-master (1)\hw-memory-master\benchmarks\objdump-D\-dmanytest.asm"
GuGuJi_fp = r"/Users/gugujixiao/allsync/hw_benchmark/LBM/try.asm"
GuGuJi_spec_fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/hw-memory/benchmarks/final_benchmark/spec_benchmark.asm"
GuGuJi_oldfp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/hw-memory/benchmarks/old_benchmark/spec_example/spec2006_470.lbm/lbm_part.asm"
fp = GuGuJi_spec_fp

In [2]:
'''读指令, 指令对象化'''
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

        

for i in temp_stat:
     if i.is_ls:
        print(i.tokens,i.ls_reg_target,i.ls_reg_target_list,i.ls_addr_offset)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
print()

('400624', 'a9bf7bfd', 'stp', None, 'x29, x30, [sp, #-16]!') None [] 0
('400630', 'a8c17bfd', 'ldp', None, 'x29, x30, [sp], #16') None [] 0
('400640', 'a9bf7bf0', 'stp', None, 'x16, x30, [sp, #-16]!') None [] 0
('400648', 'f947fe11', 'ldr', None, 'x17, [x16, #4088]') x16 [] 4088
('400664', 'f9400211', 'ldr', None, 'x17, [x16]') x16 [] 0
('400674', 'f9400611', 'ldr', None, 'x17, [x16, #8]') x16 [] 8
('400684', 'f9400a11', 'ldr', None, 'x17, [x16, #16]') x16 [] 16
('400694', 'f9400e11', 'ldr', None, 'x17, [x16, #24]') x16 [] 24
('4006a4', 'f9401211', 'ldr', None, 'x17, [x16, #32]') x16 [] 32
('4006b4', 'f9401611', 'ldr', None, 'x17, [x16, #40]') x16 [] 40
('4006c4', 'f9401a11', 'ldr', None, 'x17, [x16, #48]') x16 [] 48
('4006d4', 'f9401e11', 'ldr', None, 'x17, [x16, #56]') x16 [] 56
('4006e4', 'f9402211', 'ldr', None, 'x17, [x16, #64]') x16 [] 64
('4006f4', 'f9402611', 'ldr', None, 'x17, [x16, #72]') x16 [] 72
('400710', 'f94003e1', 'ldr', None, 'x1, [sp]') sp [] 0
('400748', 'f947ec00',

In [3]:
'''分proc'''
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
'''处理call_graph'''
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
'''生成TCFG和loop信息'''
from newCFG.cfg import TCfg
import random

tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()

for l in tcfg.loops:
    print([n.name for n in l.all_nodes])
    l.back_edge.is_backEdge = True
for n in tcfg.all_nodes:
    temp = n.inside_loop.name if n.inside_loop is not None else 'None'
    print("{} {}".format(n.name, temp))



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




['n96', 'n76', 'n78', 'n77']
['n96', 'n76', 'n78', 'n77']
['n61', 'n62']
['n63', 'n64', 'n62', 'n60', 'n61']
['n96', 'n76', 'n78', 'n77']
['n61', 'n62']
['n63', 'n64', 'n62', 'n60', 'n61']
['n54', 'n53']
['n96', 'n76', 'n78', 'n77']
['n61', 'n62']
['n63', 'n64', 'n62', 'n60', 'n61']
['n54', 'n53']
['n44', 'n43', 'n42', 'n41', 'n40']
['n96', 'n76', 'n78', 'n77']
['n61', 'n62']
['n63', 'n64', 'n62', 'n60', 'n61']
['n54', 'n53']
['n44', 'n43', 'n42', 'n41', 'n40']
['n30', 'n33', 'n34', 'n90', 'n31', 'n32']
['n96', 'n76', 'n78', 'n77']
['n61', 'n62']
['n63', 'n64', 'n62', 'n60', 'n61']
['n54', 'n53']
['n44', 'n43', 'n42', 'n41', 'n40']
['n30', 'n33', 'n34', 'n90', 'n31', 'n32']
['n23', 'n22', 'n89', 'n24', 'n20', 'n21']
['n80', 'n79', 'n78', 'n96', 'n76', 'n77', 'n75']
['n66', 'n63', 'n64', 'n62', 'n60', 'n61', 'n59', 'n65']
['n55', 'n53', 'n54', 'n52', 'n56']
['n45', 'n44', 'n43', 'n42', 'n41', 'n40', 'n39', 'n46']
['n33', 'n30', 'n35', 'n29', 'n36', 'n34', 'n90', 'n31', 'n32']
['n23', 'n

In [6]:
'''设置node和edge上的的value'''

for l in tcfg.loops:
    l.back_edge.is_backEdge = True
for n in tcfg_nodes:
    n.set_rw_data()

temp_bool = True

while temp_bool:
    no_end = False

    for n in tcfg_nodes:
        n.set_rw_value()
        if not n.no_out:
            no_end = True
        temp_bool = no_end

for e in tcfg_edges:
    if e.is_backEdge:
        e.loop_value = random.randint(0,10)
        e.edge_value = 2

for e in tcfg_edges:
    print(e.src.name,e.dst.name,e.edge_value)

n0 n12 1.0
n17 n1 1.0
n1 n4 0.5
n1 n2 0.5
n2 n18 0.5
n27 n3 0.25
n3 n28 0.25
n37 n4 0.125
n4 n38 0.625
n50 n5 0.3125
n5 n9 0.15625
n5 n6 0.15625
n6 n51 0.15625
n57 n7 0.078125
n7 n58 0.078125
n67 n8 0.0390625
n8 n9 0.0390625
n9 n68 0.1953125
n71 n10 0.1953125
n10 n72 0.1953125
n83 n11 0.09765625
n12 n84 1.0
n84 n13 1.0
n13 n85 1.0
n85 n14 1.0
n14 n86 1.0
n86 n15 1.0
n15 n87 1.0
n87 n16 1.0
n16 n88 1.0
n88 n17 1.0
n18 n26 0.5
n19 n24 0.25
n20 n89 0.125
n89 n21 0.125
n21 n23 0.0625
n21 n22 0.0625
n22 n23 0.0625
n23 n24 2
n24 n20 0.125
n24 n25 0.125
n25 n26 2
n26 n19 0.25
n26 n27 0.25
n28 n36 0.25
n29 n34 0.125
n30 n90 0.0625
n90 n31 0.0625
n31 n33 0.03125
n31 n32 0.03125
n32 n33 0.03125
n33 n34 2
n34 n30 0.0625
n34 n35 0.0625
n35 n36 2
n36 n29 0.125
n36 n37 0.125
n38 n46 0.625
n39 n44 0.3125
n40 n42 0.078125
n40 n41 0.078125
n41 n43 0.078125
n42 n43 0.078125
n43 n44 2
n44 n40 0.15625
n44 n45 0.15625
n45 n46 2
n46 n39 0.3125
n46 n47 0.3125
n47 n49 0.15625
n47 n48 0.15625
n48 n50 0.15625
n

In [7]:
'''ls分析和读写特征'''

from newCFG.cfg import TCfg
from newCFG.loadstore import LSProc
from newCFG.rw_condition import RWProc, RWType

lsproc = LSProc(tcfg_nodes)

lds_table = lsproc.ls_table

for i in lds_table:
    i.final_addr
    i.local_offset

#for i in lds_table:
#    if i.target_num == 1:
 #       pass
        #print(i.ins.tokens,i.reg_target,i.addr_offset) 
 #   else:
        #print(i.ins.tokens,i.reg_target_list[0],i.reg_target_list[1],i.addr_offset,i.final_addr)

rwproc = RWProc(lds_table)

Global_Tolerant_value = 0
Global_Intolerant_value = 0

for rwu in rwproc.rw_table:
    #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
    if rwu.is_torrent == RWType.Global_Tolerant:
        Global_Tolerant_value += rwu.find_cycle
        print(rwu.ins.tokens,rwu.ins.final_addr,rwu.find_cycle)
    if rwu.is_torrent == RWType.Global_Intolerant:
        Global_Intolerant_value += rwu.find_cycle
        #print(rwu.ins.tokens,rwu.ins.final_addr)

print("全局的容错路径为：",Global_Tolerant_value)
print("全局的非容错路径为：",Global_Intolerant_value)   

('4008fc', 'b8607840', 'ldr', None, 'w0, [x2, x0, lsl #2]') 4325656 1.5
('400930', 'b8607841', 'ldr', None, 'w1, [x2, x0, lsl #2]') 4325656 1.625
('400970', 'b8607840', 'ldr', None, 'w0, [x2, x0, lsl #2]') 4325656 1.3125
('400a90', 'b8607840', 'ldr', None, 'w0, [x2, x0, lsl #2]') 4362520 0.75
('400ac4', 'b8607841', 'ldr', None, 'w1, [x2, x0, lsl #2]') 4362520 0.8125
('400b04', 'b8607840', 'ldr', None, 'w0, [x2, x0, lsl #2]') 4362520 0.65625
('400f9c', 'b9419400', 'ldr', None, 'w0, [x0, #404]') 4325884 0.5859375
('400fbc', 'b9419400', 'ldr', None, 'w0, [x0, #404]') 4325884 0.5859375
('400fcc', 'b9419400', 'ldr', None, 'w0, [x0, #404]') 4325884 0.78125
('40100c', 'b9432400', 'ldr', None, 'w0, [x0, #804]') 4326284 0.5859375
('40101c', 'b9432400', 'ldr', None, 'w0, [x0, #804]') 4326284 0.78125
('40103c', 'b9432400', 'ldr', None, 'w0, [x0, #804]') 4326284 0.5859375
('40105c', 'b9419400', 'ldr', None, 'w0, [x0, #404]') 4325884 7.03125
('40107c', 'b9419800', 'ldr', None, 'w0, [x0, #408]') 432

In [8]:
'''cache分析'''

#打印给cache分许需要的数据
for i in lds_table:
    print("指令",i.ins.tokens,"是否是SP",i.is_sp,"指令地址",i.ins.addr.val(),"访存地址",i.final_addr,"数据宽度",i.ins.ls_data_width,"是否是加入一个数组",i.ins.is_data_group)


指令 ('40080c', 'b9001fe0', 'str', None, 'w0, [sp, #28]') 是否是SP True 指令地址 4196364 访存地址 0 数据宽度 32 是否是加入一个数组 False
指令 ('400810', 'f9000be1', 'str', None, 'x1, [sp, #16]') 是否是SP True 指令地址 4196368 访存地址 0 数据宽度 32 是否是加入一个数组 False
指令 ('400818', 'b9002fff', 'str', None, 'wzr, [sp, #44]') 是否是SP True 指令地址 4196376 访存地址 0 数据宽度 32 是否是加入一个数组 False
指令 ('400828', 'b95c3c00', 'ldr', None, 'w0, [x0, #7228]') 是否是SP False 指令地址 4196392 访存地址 4332716 数据宽度 32 是否是加入一个数组 False
指令 ('400840', 'b9002fe0', 'str', None, 'w0, [sp, #44]') 是否是SP True 指令地址 4196416 访存地址 0 数据宽度 32 是否是加入一个数组 False
指令 ('400844', 'b9402fe0', 'ldr', None, 'w0, [sp, #44]') 是否是SP True 指令地址 4196420 访存地址 0 数据宽度 32 是否是加入一个数组 False
指令 ('400854', 'b9002fe0', 'str', None, 'w0, [sp, #44]') 是否是SP True 指令地址 4196436 访存地址 0 数据宽度 32 是否是加入一个数组 False
指令 ('400868', 'b9000001', 'str', None, 'w1, [x0]') 是否是SP False 指令地址 4196456 访存地址 4445480 数据宽度 32 是否是加入一个数组 False
指令 ('400ef0', 'f9008fff', 'str', None, 'xzr, [sp, #280]') 是否是SP True 指令地址 4198128 访存地址 0 数据宽度 32 是否是

In [9]:
#from cache_analysis.read_segment import segmentReader
#from newCFG.heat_analysis import loop_heat
#bread = segmentReader(r'C:\Users\13377\Desktop\hw-memory-master (1)\hw-memory-master\benchmarks\objdump-D\-D manytest.asm')
#test=loop_heat(tcfg,lsproc,bread)
#test.loop_heat_analysis()



In [10]:
from cache_analysis.read_segment import segmentReader
from newCFG.rw_condition import RWProc, RWType
from newCFG.rw_condition_out import RWOut_Proc

guguji_segfp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/hw-memory/benchmarks/final_benchmark/spec_benchD.asm"
segreader = segmentReader(guguji_segfp)
bss_res = segreader.getbss()
data_res = segreader.getdata()

rwout = RWOut_Proc(tcfg_nodes,segreader)






for rwu in rwproc.rw_table:
    if rwu.is_torrent == RWType.Global_Tolerant:
        if rwu.ins.is_data_group:
            print(rwu.ins.final_addr,rwu.find_cycle,rwu.node.name)
    if rwu.is_torrent == RWType.Global_Intolerant:
        pass#print(rwu.ins.tokens,rwu.ins.final_addr)



4325472 4325480
4325480 4365480
4365480 4405480
4405480 4445480
4445480 4445484
4325456 4325464
4325464 4325468


TypeError: 'TCfg' object is not iterable